# Importing Libraries and dataset

In [ ]:
!pip uninstall tensorflow
!pip install tensorflow

Found existing installation: tensorflow 2.12.0
Uninstalling tensorflow-2.12.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.9/dist-packages/tensorflow-2.12.0.dist-info/*
    /usr/local/lib/python3.9/dist-packages/tensorflow/*
Proceed (Y/n)? Y
Y
  Successfully uninstalled tensorflow-2.12.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving news.csv to news.csv


In [ ]:
# Reading the data
data = pd.read_csv("news.csv")
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


# Preprocessing Dataset

In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
  title.append(data['title'][x])
  text.append(data['text'][x])
  labels.append(data['label'][x])

In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(sequences1,  padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

# Generating Word Embeddings

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving glove.6B.50d.txt to glove.6B.50d.txt


In [ ]:
embeddings_index = {}
with open('glove.6B.50d.txt') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
  
# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embeddings_matrix[i] = embedding_vector

# Model Architecture

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                              input_length=max_length, weights=[
                                  embeddings_matrix],
                              trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [ ]:
num_epochs = 50
  
training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)
  
history = model.fit(training_padded, training_labels, 
                    epochs=num_epochs,
                    validation_data=(testing_padded,
                                     testing_labels), 
                    verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50


/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 13s - loss: 0.6403 - accuracy: 0.6022 - val_loss: 0.5587 - val_accuracy: 0.6667 - 13s/epoch - 5ms/sample
Epoch 2/50
2700/2700 - 2s - loss: 0.5839 - accuracy: 0.6922 - val_loss: 0.5468 - val_accuracy: 0.6933 - 2s/epoch - 890us/sample
Epoch 3/50
2700/2700 - 3s - loss: 0.5404 - accuracy: 0.7289 - val_loss: 0.5037 - val_accuracy: 0.7067 - 3s/epoch - 1ms/sample
Epoch 4/50
2700/2700 - 3s - loss: 0.5029 - accuracy: 0.7563 - val_loss: 0.4953 - val_accuracy: 0.7700 - 3s/epoch - 1ms/sample
Epoch 5/50
2700/2700 - 3s - loss: 0.4584 - accuracy: 0.7781 - val_loss: 0.4828 - val_accuracy: 0.7267 - 3s/epoch - 1ms/sample
Epoch 6/50
2700/2700 - 3s - loss: 0.3997 - accuracy: 0.8159 - val_loss: 0.5099 - val_accuracy: 0.7500 - 3s/epoch - 960us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3965 - accuracy: 0.8181 - val_loss: 0.4933 - val_accuracy: 0.7567 - 2s/epoch - 868us/sample
Epoch 8/50
2700/2700 - 3s - loss: 0.3275 - accuracy: 0.8585 - val_loss: 0.5828 - val_accuracy: 0.7367 - 3s/epoch - 993us/

# Model Evaluation and Predeiction

In [ ]:
# sample text to check if fake or not
X = "Karry to go to France in gesture of sympathy"
  
# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type, 
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
  print("This news is True")
else:
  print("This news is false")

This news is false
